# library

In [41]:
from Cryptodome.Cipher import AES  # from pycryptodomex v-3.10.4
from Cryptodome.Random import get_random_bytes

import hashlib
import binascii

# fungsi kripto

In [42]:
HASH_NAME = "SHA256"
IV_LENGTH = 16
ITERATION_COUNT = 989898
KEY_LENGTH = 32

In [43]:
def pad(message):
    padding_length = AES.block_size - len(message) % AES.block_size #panjang padding adalah selisih dari antara ukuran blok (16) dengan panjang pesan modulo blok (< 16. jika sama dengan 16 maka tidak ada padding)
    padding = bytes([padding_length]) * padding_length #misalnya nilai padding length adalah 14, maka paddingnya adalah bentuk byte dari 14 sebanyak 14 kali
    return message + padding

In [44]:
def unpad(message):
    padding_length = message[-1] #mengambil byte paling ujung (sebagai contoh byte 14)
    return message[:-padding_length] #dan menghapus padding sesuai dengan padding length yang telah diketahui

In [45]:
def get_secret_key(password, salt):
    return hashlib.pbkdf2_hmac(HASH_NAME, password.encode(), salt.encode(), ITERATION_COUNT, KEY_LENGTH)

In [46]:
def encrypt(password, salt, message):
    secret = get_secret_key(password, salt)
    message = pad(message.encode("utf8"))
    iv = get_random_bytes(IV_LENGTH)
    cipher = AES.new(secret, AES.MODE_CBC, iv)
    cipher_bytes = iv + cipher.encrypt(message)
    ciphertext_hex = binascii.hexlify(cipher_bytes).decode()
    return ciphertext_hex

In [47]:
def decrypt(password, salt, ciphertext_hex):
    secret = get_secret_key(password, salt)
    ciphertext = bytes.fromhex(ciphertext_hex)
    iv = ciphertext[:IV_LENGTH]
    cipher = AES.new(secret, AES.MODE_CBC, iv)
    plaintext_bytes = unpad(cipher.decrypt(ciphertext[IV_LENGTH:]))
    plaintext = plaintext_bytes.decode("utf8")
    return plaintext

In [48]:
garam = "aqua_aria_marine"
password = "ini password aquamarine"

In [49]:
filename = "testing.txt"
output_filename = "testing-ciphertext-original.txt"
output_filename2 = "testing-plaintext-decoded-original.txt"

# enkripsi (ubah jumlah iteration count)

In [50]:
file = open(filename,"r",encoding='utf8')
message = (file.read())
file.close() #do not forget to close

In [51]:
ciphertext_hex = encrypt(password, garam, message)

In [52]:
with open(output_filename, "w",encoding='utf8') as file:
    file.write(ciphertext_hex)

# dekripsi

In [56]:
with open(output_filename, "r",encoding='utf8') as file:
    ciphertext_hex = file.read().strip()

In [57]:
plaintext = decrypt(password, garam, ciphertext_hex)

In [58]:
with open(output_filename2, "w",encoding='utf8') as file:
    file.write(plaintext)

# steganografi original (gunakan ini jika message grids > 64)

In [59]:
import bpcs

In [60]:
#threshold1 = 0.3
#threshold2 = 0.4
threshold3 = 0.5

embedded_file = "testing-ciphertext-original.txt"
cover1 = 'cover/cover-1.png'
stegoimage1_3 = 'pengujian/stego/pan-dog/stego3.png'
embedded_file_from_stegoimage1_3 = "pengujian/stego/pan-dog/testing-ciphertext-original3.txt"

In [20]:
bpcs.encode(cover1,embedded_file,stegoimage1_3,threshold3)

Berhasil memasukkan 1068 grid pesan asli dan 34 map konjugat


In [23]:
bpcs.decode(stegoimage1_3,embedded_file_from_stegoimage1_3,threshold3)

Ditemukan 1102 dari 98304 grid dengan threshold kompleksitas di atas 0.5
Ditemukan 1068 grid pesan dan 34 map konjugat


# METADATA MP3

In [61]:
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC

In [63]:
def remove_all_metadata(mp3_file):
    # Buka berkas MP3
    audio = MP3(mp3_file)

    # Hapus seluruh metadata ID3 tag
    audio.delete()

    # Simpan perubahan ke berkas MP3
    audio.save()

# Contoh pemanggilan fungsi
mp3_file_path = "pengujian/mp3/audio.mp3"
remove_all_metadata(mp3_file_path)

APIC TYPE GUIDANCE :

1 = ICON

2 = ICON JUGA

3 = FRONT COVER

4 = BACK COVER

5 = LEAFLET

APIC TYPE BISA DARI 0 SAMPE 20, TAPI 1 SAMPE 5 YANG MUDAH DIBACA SISTEM DAN SOFTWARE WALAU KADANG GAK KEBACA SAMA PLAYER

In [64]:
def embed_cover_image(mp3_file, image_file):
    audio = ID3(mp3_file)

    # Baca gambar sebagai bytes
    with open(image_file, "rb") as img_file:
        image_data = img_file.read()

    # Tambahkan gambar sebagai cover album
    audio["APIC"] = APIC(
        encoding=3,  # Encoding untuk gambar adalah utf-8
        mime="image/png",  # Ganti sesuai tipe gambar yang digunakan (jpeg, png, dll.), APIC-1
        type=4,  #back cover, APIC-2
        desc="You don't expect this as stego image, right?",  # Deskripsi cover album, APIC-3
        data=image_data  # Data gambar dalam bentuk bytes
    )

    # Save perubahan MP3
    audio.save()

mp3_file_path = "pengujian/mp3/audio.mp3"
image_file_path = "pengujian/mp3/stego3.png"
embed_cover_image(mp3_file_path, image_file_path)

In [65]:
def embed_cover_image(mp3_file, image_file):
    audio = ID3(mp3_file)

    # Baca gambar sebagai bytes
    with open(image_file, "rb") as img_file:
        image_data = img_file.read()

    #TAG editing
    audio["APIC"] = APIC(
        encoding=3,  #UTF-8 biar pure
        mime="image/png",  # tipe, png atau jpeg
        type=2,  #icon
        desc="This is not stego image... perhaps",  # Deskripsi
        data=image_data
    )

    # Save perubahan MP3
    audio.save()

mp3_file_path = "pengujian/mp3/audio.mp3"
image_file_path = "pengujian/mp3/stego3-decoy message.png"
embed_cover_image(mp3_file_path, image_file_path)

In [66]:
def extract_specific(mp3_file, output_path):
    audio = ID3(mp3_file_path)
    apic_frames = audio.getall("APIC") #get semua APIC yang tersemat
    for apic_frame in apic_frames:
        if apic_frame.type == 4: #ganti type disini
            with open(output_path, "wb") as output_file:
                output_file.write(apic_frame.data)

mp3_file_path = "pengujian/mp3/audio.mp3"
output_image_file_path = "pengujian/mp3/realstego3.png"

extract_specific(mp3_file_path,output_image_file_path)

In [67]:
message_extracted_from_mp3 = "pengujian/mp3/testing-ciphertext-original.txt"

# ekstraksi pesan dari citra, dekripsi

In [68]:
bpcs.decode(output_image_file_path,message_extracted_from_mp3,threshold3)

Ditemukan 1102 dari 98304 grid dengan threshold kompleksitas di atas 0.5
Ditemukan 1068 grid pesan dan 34 map konjugat


In [69]:
import re

with open(message_extracted_from_mp3, "r") as file:
    cipher = file.read()
    ciphertext_hex_cleaned = re.sub(r'[^0-9a-fA-F]', '', cipher)

In [70]:
plaintext_mp3 = decrypt(password, garam, ciphertext_hex_cleaned)

In [71]:
with open ("pengujian/mp3/plaintext_from_mp3.txt", "w") as file :
    file.write(plaintext_mp3)